## Installation des librairies nécessaires

In [1]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow_text
!pip install spacy
!pip install kss # Outil pour la segmentation des phrases coréennes : https://github.com/hyunwoongko/kss

## Installation des librairies nécessaires pour le coréen

In [2]:
# Installation de MeCab
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!pip install mecab-python

# Installation de KoNLPy
!pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

!echo `mecab-config --dicdir`"/mecab-ko-dic" # Vérifie le chemin du dictionnaire

# Initialisation de Mecab avec le chemin du dictionnaire
from konlpy.tag import Mecab
dicpath = "/usr/local/lib/mecab/dic/mecab-ko-dic"
mecab = Mecab(dicpath)

# Test
#text = "안녕하세요, 반갑습니다."
#print("Mecab:", mecab.morphs(text))

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
curl is already the newest version (7.81.0-1ubuntu1.15).
python3-dev is already the newest version (3.10.6-1~22.04).
openjdk-8-jdk is already the newest version (8u392-ga-1~22.04).
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
/usr/local/lib/mecab/dic/mecab-ko-dic


## Fonctions

In [3]:
from spacy.lang.en import English
from spacy.lang.ko import Korean
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import kss

In [116]:
from konlpy.tag import Mecab
import kss

def sentencize(text, lang):
    """
    Sépare le texte d'entrée en phrases en fonction de la langue spécifiée.

    Parameters :
    - text (str) : Le texte d'entrée à tokeniser en phrases.
    - lang (str) : La langue du texte d'entrée ("en" pour l'anglais, "ko" pour le coréen).

    Returns :
    - list de str : Une liste de phrases extraites du texte d'entrée.

    Raises :
    - ValueError : Si la langue spécifiée n'est pas prise en charge.

    Note :
    - Pour le texte en anglais, le sentencizer de spaCy est utilisé.
    - Pour le texte en coréen, Mecab est utilisé pour l'analyse morphologique et kss est utilisé pour la séparation des phrases.
    """
    sents = []

    if lang == "en":
        nlp = English()
        nlp.add_pipe("sentencizer")
        doc = nlp(text)
        for sent in doc.sents:
            sents.append(sent.text.replace('\n', ' ').strip())

    elif lang == "ko":
        mecab = Mecab(dicpath)
        doc = kss.split_sentences(text)
        sents = [sent.replace('\n', ' ').strip() for sent in doc]

    else:
        raise ValueError("Langue non prise en charge.")

    return sents

In [117]:
def align(en_sents, ko_sents):
    """
    Fonction pour aligner les phrases en anglais et coréen en utilisant Universal Sentence Encoder.

    Parameters:
    - en_sents (list): Liste des phrases en anglais.
    - ko_sents (list): Liste des phrases en coréen.

    Returns:
    - list: Liste des paires de phrases alignées.
    """
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    en_result = embed(en_sents)
    ko_result = embed(ko_sents)
    sims = np.inner(en_result, ko_result)

    costs = np.zeros((len(en_sents)+1, len(ko_sents)+1))
    pointers = np.zeros((len(en_sents)+1, len(ko_sents)+1), dtype=int)

    for i in range(1, len(en_sents)+1):
        costs[i, 0] = costs[i-1, 0] + 1.

    for j in range(1, len(ko_sents)+1):
        costs[0, j] = costs[0, j-1] + 1.

    for i in range(1, len(en_sents)+1):
        for j in range(1, len(ko_sents)+1):
            choices = [
                (costs[i-1, j-1] + (1. - sims[i-1, j-1]), 1),
                (costs[i-1, j] + 1., 2),
                (costs[i, j-1] + 1., 3)
            ]
            best_choice = sorted(choices, key=lambda x: x[0])[0]
            costs[i, j], pointers[i, j] = best_choice

    aligned = []
    i, j = len(en_sents), len(ko_sents)
    while i > 0 or j > 0:
        if pointers[i, j] == 1:
            i -= 1
            j -= 1
            aligned.append((en_sents[i], ko_sents[j]))
        elif pointers[i, j] == 2:
            i -= 1
            aligned.append((en_sents[i], ''))
        elif pointers[i, j] == 3:
            j -= 1
            aligned.append(('', ko_sents[j]))

    aligned.reverse()

    return aligned

In [118]:
def align_and_filter(en_sents, ko_sents):
    """
    Aligne et filtre les paires de phrases en anglais et coréen.

    Parameters :
    - en_sents (list de str) : Liste des phrases en anglais.
    - ko_sents (list de str) : Liste des phrases en coréen.

    Returns :
    - list de tuple : Liste des paires de phrases alignées et filtrées.

    Note :
    - L'alignement est gérée par la fonction 'align'.
    - Seules les paires de phrases qui ont une correspondance en anglais et en coréen sont conservées.
    """
    return [(en, ko) for en, ko in align(en_sents, ko_sents) if en and ko]


## Importation fichiers à traiter et éxecution fonctions

In [119]:
from google.colab import drive
drive.mount('/content/drive')

# Accéder aux fichiers dans le dossier 'My Drive' de Google Drive
# Mettre les fichiers dans le même dossier que le notebook
en_path = '/content/drive/My Drive/Colab Notebooks/test_en.txt'
ko_path = '/content/drive/My Drive/Colab Notebooks/gen_ko.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
# Lire le contenu des fichiers
with open(en_path, 'r', encoding='utf-8') as file_en:
    en_lines = file_en.read()

with open(ko_path, 'r', encoding='utf-8') as file_ko:
    ko_lines = file_ko.read()

In [122]:
en_sents = sentencize(en_lines, "en")
ko_sents = sentencize(ko_lines, "ko")

# Affichez le nombre total de phrases coréennes
print("Nombre total de phrases coréennes:", len(ko_sents))

Nombre total de phrases coréennes: 1371


In [123]:
# Alignement et filtrage des phrases
aligned_and_filtered_sentences = align_and_filter(en_sents, ko_sents)

## Enregistrement du résultat
### Un fichier par langue

In [18]:
resultat_en_path = '/content/drive/My Drive/Colab Notebooks/test_resultat_en.txt'
resultat_ko_path = '/content/drive/My Drive/Colab Notebooks/test_resultat_ko.txt'

with open(resultat_en_path, "w", encoding="utf-8") as file_en:
    with open(resultat_ko_path, "w", encoding="utf-8") as file_ko:
        for en_sent, ko_sent in aligned_and_filtered_sentences:
            file_en.write(en_sent + "\n")
            file_ko.write(ko_sent + "\n")